In [ ]:
import pandas as pd
import numpy as np
import pickle as pkl
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

ratings_df = pd.read_csv("bbalanced_simulated_ratings_50_users_renamed.csv")
exhibits_df = pd.read_excel("data set - New.xlsx")

ratings_df = ratings_df.dropna(subset=['user_id', 'exhibit_id', 'rating'])
ratings_df['rating'] = ratings_df['rating'].astype(int)

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
ratings_df['user_id_encoded'] = user_encoder.fit_transform(ratings_df['user_id'])
ratings_df['item_id_encoded'] = item_encoder.fit_transform(ratings_df['exhibit_id'])

num_users = ratings_df['user_id_encoded'].nunique()
num_items = ratings_df['item_id_encoded'].nunique()

train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42, stratify=ratings_df['user_id_encoded'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

def prepare_inputs_cf(data):
    return [np.array(data['user_id_encoded']), np.array(data['item_id_encoded'])], np.array(data['rating']).astype('float32')

train_inputs, train_labels = prepare_inputs_cf(train_df)
val_inputs, val_labels = prepare_inputs_cf(val_df)
test_inputs, test_labels = prepare_inputs_cf(test_df)

user_input = Input(shape=(1,))
item_input = Input(shape=(1,))
user_emb = Flatten()(Embedding(num_users, 64)(user_input))
item_emb = Flatten()(Embedding(num_items, 64)(item_input))
x = Concatenate()([user_emb, item_emb])
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model_cf = Model([user_input, item_input], output)
model_cf.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

model_cf.fit(train_inputs, train_labels, validation_data=(val_inputs, val_labels), epochs=50, batch_size=32)

y_probs_cf = model_cf.predict(test_inputs)
y_preds_cf = (y_probs_cf >= 0.5).astype(int).flatten()

def evaluate(y_true, y_pred, y_prob):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-10)
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"AUC:       {auc:.4f}")
    print(f"F1 Score:  {f1:.4f}")

evaluate(test_labels, y_preds_cf, y_probs_cf)


In [ ]:
model_cf.save("model_collaborative.h5")

In [ ]:
import pandas as pd
import numpy as np
import pickle as pkl
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

ratings_df = pd.read_csv("bbalanced_simulated_ratings_50_users_renamed.csv")
exhibits_df = pd.read_excel("data set - New.xlsx")
with open("all_image_features1.pkl", "rb") as f:
    image_features_dict = pkl.load(f)

exhibits_df = exhibits_df[['Exhibit_id', 'exhibits', 'Text in English', 'Category', 'Time Period']].dropna()

image_features = []
for name in exhibits_df['exhibits']:
    if name in image_features_dict and len(image_features_dict[name]) > 0:
        vectors = [entry['feature'] for entry in image_features_dict[name]]
        avg_vector = np.mean(np.stack(vectors), axis=0)
    else:
        avg_vector = np.zeros(2048)
    image_features.append(avg_vector)
image_features = np.array(image_features)
image_features /= np.linalg.norm(image_features, axis=1, keepdims=True) + 1e-10

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
texts = exhibits_df['Text in English'].astype(str).tolist()
tokens = tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors="tf")
bert_output = bert_model(input_ids=tokens['input_ids'], attention_mask=tokens['attention_mask'])
text_embeddings = bert_output.last_hidden_state[:, 0, :].numpy()

encoder = OneHotEncoder(sparse_output=False)
category_encoded = encoder.fit_transform(exhibits_df[['Category']])
time_encoded = encoder.fit_transform(exhibits_df[['Time Period']])

combined_features = np.concatenate([text_embeddings, category_encoded, time_encoded, image_features], axis=1)

exhibit_to_vector = dict(zip(exhibits_df['Exhibit_id'], combined_features))
ratings_df['features'] = ratings_df['exhibit_id'].map(exhibit_to_vector)
ratings_df.dropna(subset=['features'], inplace=True)

train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42, stratify=ratings_df['user_id'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

def prepare_inputs_cb(data):
    return np.stack(data['features'].values), np.array(data['rating']).astype('float32')

train_inputs, train_labels = prepare_inputs_cb(train_df)
val_inputs, val_labels = prepare_inputs_cb(val_df)
test_inputs, test_labels = prepare_inputs_cb(test_df)

input_features = Input(shape=(combined_features.shape[1],))
x = Dense(256, activation='relu')(input_features)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model_cb = Model(input_features, output)
model_cb.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

# --- 9. Train ---
model_cb.fit(train_inputs, train_labels, validation_data=(val_inputs, val_labels), epochs=50, batch_size=32)

# --- 10. Evaluate ---
y_probs_cb = model_cb.predict(test_inputs)
y_preds_cb = (y_probs_cb >= 0.5).astype(int).flatten()

evaluate(test_labels, y_preds_cb, y_probs_cb)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.5426 - loss: 0.6882 - val_accuracy: 0.5688 - val_loss: 0.6717
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6147 - loss: 0.6648 - val_accuracy: 0.6125 - val_loss: 0.6806
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6477 - loss: 0.6389 - val_accuracy: 0.6125 - val_loss: 0.6761
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6829 - loss: 0.6131 - val_accuracy: 0.6000 - val_loss: 0.6838
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6413 - loss: 0.6436 - val_accuracy: 0.5750 - val_loss: 0.7137
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6369 - loss: 0.6389 - val_accuracy: 0.5875 - val_loss: 0.7086
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7040 - loss: 0.5958 - val_accuracy: 0.5750 - val_loss: 0.7089
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6966 - loss: 0.6015 - val_accuracy: 0.6250 - v

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Accuracy:  0.5800
Precision: 0.5714
Recall:    0.5361
AUC:       0.6244
F1 Score:  0.5532


In [ ]:
model_cb.save("model_content_based.h5")


In [ ]:
from tensorflow.keras.models import load_model

# Load models
model_cf = load_model("model_collaborative.h5")
model_cb = load_model("model_content_based.h5")
